In [18]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image
import io
import torch.nn.functional as F
import nest_asyncio
import uvicorn

# Apply fix for Jupyter Notebook
nest_asyncio.apply()

app = FastAPI()

# Initialize MTCNN and FaceNet models
mtcnn = MTCNN(keep_all=True, min_face_size=40, thresholds=[0.6, 0.7, 0.7])
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def read_image(file: UploadFile):
    image = Image.open(io.BytesIO(file.file.read())).convert("RGB")
    
    # Convert to numpy array
    image_np = np.array(image)

    # Apply histogram equalization to enhance contrast
    img_yuv = cv2.cvtColor(image_np, cv2.COLOR_RGB2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    image_np = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

    return image_np

def extract_face(image):
    face_box, _ = mtcnn.detect(image)
    if face_box is None:
        raise HTTPException(status_code=400, detail="No face detected")

    x1, y1, x2, y2 = map(int, face_box[0])
    face = image[y1:y2, x1:x2]

    # Resize the face to 160x160
    face = cv2.resize(face, (160, 160))

    return face


def get_embedding(face):
    face = Image.fromarray(face).resize((160, 160))  # Resize for FaceNet
    face = np.array(face).astype(np.float32)
    face = (face - 127.5) / 128.0  # Normalize
    face = np.transpose(face, (2, 0, 1))
    face = torch.tensor(face).unsqueeze(0)

    with torch.no_grad():
        embedding = resnet(face)

    norm = torch.norm(embedding)
    if norm == 0:
        raise HTTPException(status_code=400, detail="Failed to compute face embedding")
    
    return embedding / norm  # Normalize the embedding

def cosine_similarity(embedding1, embedding2):
    """ Computes both Cosine Similarity and Euclidean Distance """
    cos_sim = F.cosine_similarity(embedding1, embedding2).item()
    euclidean_dist = torch.dist(embedding1, embedding2, p=2).item()

    # Match condition: High cosine similarity + Low Euclidean distance
    match = (cos_sim > 0.65) and (euclidean_dist < 1.0)

    return match, cos_sim  # Return both match status and score



@app.post("/verify-face")
async def verify_face(selfie: UploadFile = File(...), document: UploadFile = File(...)):
    print(f"Received selfie: {selfie.filename}, document: {document.filename}")
    
    try:
        selfie_image = read_image(selfie)
        document_image = read_image(document)

        print("Extracting faces...")
        selfie_face = extract_face(selfie_image)
        document_face = extract_face(document_image)

        print("Computing embeddings...")
        selfie_embedding = get_embedding(selfie_face)
        document_embedding = get_embedding(document_face)
        

        match, similarity_score = cosine_similarity(selfie_embedding, document_embedding)

        return JSONResponse(content={
            "match": bool(match),  # Ensure match is a boolean
            "similarity_score": float(similarity_score)  # Ensure it's a float
        })


    except Exception as e:
        print("Error:", str(e))
        raise HTTPException(status_code=400, detail=str(e))

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [10068]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
import os
os._exit(0)

: 

In [ ]:
%tb